In [25]:
import requests
from bs4 import BeautifulSoup
import psycopg2

In [26]:
DB_NAME = "kitap_db"
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"

In [27]:
con = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port = DB_PORT
)
con.autocommit = True
cursor = con.cursor()

In [28]:
query_create_table = """ CREATE TABLE IF NOT EXISTS dr_kitaplar(
    id Serial PRIMARY KEY,
    kitapAdi TEXT NOT NULL,
    yazar TEXT NOT NULL,
    yayinEvi TEXT NOT NULL,
    ucret TEXT NOT NULL,
    link TEXT NOT NULL
    );
    """
cursor.execute(query_create_table)

In [29]:
def add_db_user(book_name, book_author, book_publishing, book_price, book_URL):
    query_INSERT = f"INSERT INTO dr_kitaplar (kitapAdi, yazar, yayinEvi, ucret, link) SELECT '{book_name}', '{book_author}', '{book_publishing}', '{book_price}', '{book_URL}' WHERE NOT EXISTS ( SELECT * FROM dr_kitaplar WHERE kitapAdi='{book_name}' and yazar='{book_author}' and yayinEvi='{book_publishing}' and ucret='{book_price}' and link='{book_URL}');"
    cursor.execute(query_INSERT)

In [30]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}

ProductCount = 0
for pageNumber in range(1, 4):
    r = requests.get(f"https://www.dr.com.tr/kategori/Kitap/Bilim/grupno=00052?Page={pageNumber}", headers=header)

    soup = BeautifulSoup(r.content, "lxml")

    drBooks = soup.find_all("div", attrs={"class":"prd-content"})


    for books in drBooks:
        ProductCount += 1
        book_name = books.find("a", attrs={"class": "prd-name"}).text.replace("'"," ")
        try:
            book_author = books.find("a", attrs={"class": "who text-overflow"}).text.strip().replace("'"," ")
        except Exception:
            book_author = books.find("div", attrs={"class": "prd-row"}).text.replace("'"," ")

        book_publishing = books.find("a", attrs = {"class":"prd-publisher"}).text.strip()
        book_price = books.find("div", attrs = {"class": "prd-price"}).text.strip()
        book_URL =  "https://www.dr.com.tr" + books.find("a").get("href")
            
        book = {"Kitap-Adi":book_name, "Kitap-Yazar": book_author, "Kitap-Yayin": book_publishing, "Kitap-Ucret": book_price, "Kitap-URL": book_URL}

        add_db_user(book["Kitap-Adi"], book["Kitap-Yazar"], book["Kitap-Yayin"], book["Kitap-Ucret"], book["Kitap-URL"])
print(f"toplam {ProductCount} ürün başarılı bir şekilde veri tabanına eklendi.")

toplam 45 ürün başarılı bir şekilde veri tabanına eklendi.
